# App data analyst guided project (Dataquest)

This aim of this project is to act as a data analyst for a hypothetical app company which produces free apps and therefore gets income through ad impressions. The apps are designed for an English-speaking audience. The idea is to find apps within the provided Apple and Google datasets which are well-received and generating the most interaction/views, so as to create more apps with similar traits.
There are two datasets: one from the Apple App Store ([documentation](https://www.kaggle.com/datasets/ramamet4/app-store-apple-data-set-10k-apps)), and one from the Google Play Store ([documentation](https://www.kaggle.com/datasets/lava18/google-play-store-apps)).

In [1]:
#This function was provided by Dataquest as a useful way to explore the data

def explore_data(dataset, start, end, rows_and_columns=False):

    dataset_slice = dataset[start:end]    

    for row in dataset_slice:
 
        print(row)
        print('\n') # adds a new (empty) line after each row

    if rows_and_columns:

        print('Number of rows:', len(dataset))
        print('Number of columns:', len(dataset[0]))
        
        

# This function takes in a dataset (in this case a list of lists), 
# cuts it into single entries, and then displays the 
# entries with a space in between separate entries for clarity and ease of reading.
# The 'start' and 'end' values of the slice are the indices of the selected list elements (that is, index 0 
# is the first row (often a header), and each of the subsequent indices represents one entry about a particular app).
# rows_and_columns prints further information about the structure of the dataset, if 'True'
# The dataquest instructions specify that the dataset should not have a header, or this
# function will return a row count which includes the header.

In [2]:
AppleStorecsv = open('AppleStore.csv', encoding='utf8')

GooglePlaycsv = open('googleplaystore.csv', encoding='utf8')

from csv import reader #import the reader function to generate reader objects which iterate over the datasets

apple = reader(AppleStorecsv) #generate reader objects
google = reader(GooglePlaycsv)

applestore = list(apple) #make a list of lists (each list is the data for one app, plus an entry for the header)
apple_header = applestore[0] #separate out the header, per Dataquest instructions
apple_apps = applestore[1:] #all app data

googlestore = list(google)
google_header = googlestore[0]
google_apps = googlestore[1:]

### Exploring the datasets

In [3]:
print(apple_header)
print('\n')
explore_data(apple_apps, 0, 3, True)

['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


Number of rows: 7197
Number of columns: 16


In [4]:
print(google_header)
print('\n')
explore_data(google_apps, 0, 3, True)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['Coloring book moana', 'ART_AND_DESIGN', '3.9', '967', '14M', '500,000+', 'Free', '0', 'Everyone', 'Art & Design;Pretend Play', 'January 15, 2018', '2.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


Number of rows: 10841
Number of columns: 13


The datasets have disparate numbers of columns (so different information about the apps), arranged in different orders and named in different ways.

## Data cleaning

Before looking for particular types of apps, it's important to first check the integrity of the datasets. All entries should have the same number of elements as the header (otherwise they are missing data for one column or another). The datasets contain apps in various languages, and these are superfluous to my goals. Non-free apps are also superfluous.

In [5]:
#To check if all of the entries are the same length as the header:

print("Apple dataset:", apple_header)

i = 0

for row in apple_apps:

    if len(row) != len(apple_header):

        print(row)

        print("Row " + str(apple_apps.index(row)) + ":")

        i += 1

print(i, "row(s)")
print('\n')

print("Google dataset:", google_header)

i = 0

for row in google_apps:

    if len(row) != len(google_header):

        print("Row " + str(google_apps.index(row)) + ":")
 
        print(row)
 
        i += 1

print(i, "row(s)")
        

Apple dataset: ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
0 row(s)


Google dataset: ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
Row 10472:
['Life Made WI-Fi Touchscreen Photo Frame', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', '', 'February 11, 2018', '1.0.19', '4.0 and up']
1 row(s)


A single row in the Google dataset is missing an entry. Given the structure of the header, it is clear that the missing datapoint is in the 'Category' column. The [Google Play Store page for this app](https://play.google.com/store/apps/details?id=com.lifemade.internetPhotoframe) shows that it is categorised as a "Lifestyle" app.

In [6]:
#Double-checking that 'Lifestyle' is a category used in the dataset:

categories = [] #make an empty list

for row in google_apps: #check each row of the google_apps dataset

    if row[1] not in categories: #if the second element (column 2, which has index 1) isn't present

        categories.append(row[1]) #add it to a list

print(categories) #print the list of categories

['ART_AND_DESIGN', 'AUTO_AND_VEHICLES', 'BEAUTY', 'BOOKS_AND_REFERENCE', 'BUSINESS', 'COMICS', 'COMMUNICATION', 'DATING', 'EDUCATION', 'ENTERTAINMENT', 'EVENTS', 'FINANCE', 'FOOD_AND_DRINK', 'HEALTH_AND_FITNESS', 'HOUSE_AND_HOME', 'LIBRARIES_AND_DEMO', 'LIFESTYLE', 'GAME', 'FAMILY', 'MEDICAL', 'SOCIAL', 'SHOPPING', 'PHOTOGRAPHY', 'SPORTS', 'TRAVEL_AND_LOCAL', 'TOOLS', 'PERSONALIZATION', 'PRODUCTIVITY', 'PARENTING', 'WEATHER', 'VIDEO_PLAYERS', 'NEWS_AND_MAGAZINES', 'MAPS_AND_NAVIGATION', '1.9']


This list contains a variety of categories, including 'LIFESTYLE', confirming that this category is present in the dataset. There is also an entry for '1.9'. Given that this appears to be the rating of the app with missing data, this is the expected result (a frameshift to the next column). The app's 'Genres' column has an entry, but it's blank. Scrolling through the dataset at the source makes it clear that apps with 'LIFESTYLE' in the 'Category' column have 'Lifestyle' in the 'Genres' column, 'ART_AND_DESIGN' apps have 'Art & Design' in the 'Genres' column, etc.

In [7]:
#Since the category/genre is known, fixing that entry:
if google_apps[10472][1] != 'LIFESTYLE': #avoiding accidental duplication of this command within the anaconda environment

    google_apps[10472].insert(1, 'LIFESTYLE') #insert a column for the category
    google_apps[10472][9] =  ('Lifestyle') #correct the blank column

print(google_apps[10472])

['Life Made WI-Fi Touchscreen Photo Frame', 'LIFESTYLE', '1.9', '19', '3.0M', '1,000+', 'Free', '0', 'Everyone', 'Lifestyle', 'February 11, 2018', '1.0.19', '4.0 and up']


In [8]:
#verifying that all entries are now the same length:

print("Apple dataset:", apple_header)

i = 0

for row in apple_apps:

    if len(row) != len(apple_header):
    
        print(row)
        
        print("Row " + str(apple_apps.index(row)) + ":")
        
        i += 1

print(i, "row(s)")    
print('\n')

print("Google dataset:", google_header)

i = 0

for row in google_apps:

    if len(row) != len(google_header):

        print("Row " + str(google_apps.index(row)) + ":")

        print(row)

        i += 1

print(i, "row(s)")
        

Apple dataset: ['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']
0 row(s)


Google dataset: ['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']
0 row(s)


The forums for the datasets indicate that the Google apps dataset includes duplicate entries. First I will confirm that this is true:

In [9]:
google_apps_names = []
google_duplicates = 0

for row in google_apps:

    if row[0] not in google_apps_names:
        google_apps_names.append(row[0])
    
    elif row[0] in google_apps_names:
        google_duplicates += 1

print('Duplicates:', google_duplicates)

Duplicates: 1181


Since there are quite a lot of duplicates, I will not remove them individually. However, I want to be careful to keep the newest data and discard older data. With this in mind, the number of reviews should be a good guide to the chronology of the data. Entries which have the same app name but larger numbers of reviews must be newer than entries which share the app name but have fewer recorded reviews.

In [10]:
#generating a list of all duplicates, containing just name and number of reviews

google_apps_names = [] #clear out the list to repeat above process in a modified fashion
google_dupes = [] #make a new empty list

for row in google_apps:

    if row[0] not in google_apps_names:
        google_apps_names.append(row[0]) #if the name isn't in the list, add it
    
    elif row[0] in google_apps_names:
        google_dupes.append([row[0], row[3]]) #if the name is already on the list of apps, 
                                                                                                        # add an entry to "dupes" with app name and number of reviews
        
print(google_dupes[:5])
print('Entries:', len(google_dupes))

[['Quick PDF Scanner + OCR FREE', '80805'], ['Box', '159872'], ['Google My Business', '70991'], ['ZOOM Cloud Meetings', '31614'], ['join.me - Simple Meetings', '6989']]
Entries: 1181


This appears to have worked, and the number of entries is correct (i.e. it's the same as the number of duplicate entries counted above). To remove duplicate entries non-randomly, I want to compare the number of reviews and delete all entries in google_apps except that with the highest number of reviews. Since there are 1181 duplicate entries, the expected length of the cleaned dataset is given by:

In [11]:
print('Expected length:', len(google_apps) - 1181)

Expected length: 9660


In [12]:
#first, I will create a dictionary to hold information about the maximum number of reviews received in an entry for a duplicated app:
max_reviews = {}

for app in google_dupes:

    name = app[0]
    n_reviews = float(app[1])
    
    if name in max_reviews: #if the name is in the dictionary
    
        if n_reviews > max_reviews[name]: #and if the number of reviews is greater than that already recorded
            max_reviews[name] = n_reviews #record the higher number of reviews as the dictionary entry
    
    elif name not in max_reviews:
        max_reviews[name] = n_reviews #otherwise, add a key:value pair for this app to the dictionary

In [13]:
#Now I will create a clean dataset, removing all entries which have fewer ratings than the maximum recorded number for the app

google_apps_clean = []
apps_added = []

for app in google_apps:

    name = app[0]
    n_reviews = float(app[3])

    if name in max_reviews:                       #check if the name of the app is in the dictionary of duplicates, if it is...
 
        if n_reviews == max_reviews[name]:  #check if the number of reviews equals the maximum number recorded for that app. If it is...

            if name not in apps_added:          # make sure the app isn't already in the cleaned dataset
                google_apps_clean.append(app) # add the app to the cleaned dataset
                apps_added.append(app[0])       #record that the app has been added to the cleaned dataset

    elif name not in apps_added:               # otherwise, check if the name of the app has not been added to the cleaned dataset
        google_apps_clean.append(app)         # if it hasn't, add it
        apps_added.append(app[0])               # record that it was added
    
print(len(google_apps_clean))                     #checking that the length of the cleaned dataset is as expected

9660


The number of apps in the cleaned dataset is correct.

Given that the hypothetical company produces free apps in English with in-app advertising, I want to look for English apps which are free as a point of comparison.
In my next steps, I will remove all apps from both datasets which are not:

1) in English

2) Free

First, some code to check whether a string is in English:

In [14]:
def isEnglish(a_string):    #check if a string is in English, with a bit of wiggle-room for emojis etc.
   
    i = 0                            # create an index variable
    
    for letter in a_string:   
        
        if ord(letter) > 127:   #check whether each letter is part of the English alphabet
            
            i += 1                   #if it's not, increase the index
            
            if i > 3:                 #if the index of non-English letters gets above 3, return False
                return False
        
    return True                  #if the function hasn't returned anything else, return True

In [15]:
#using the above function to create English-only datasets:
google_english = []
apple_english = []
for app in google_apps_clean:
    name = app[0]
    if isEnglish(name):                            
            google_english.append(app)           
    
for app in apple_apps:
    name = app[1]
    if isEnglish(name):
        apple_english.append(app)
    
    
print('There are ', len(google_english), 'English Google apps')
print('There are ', len(apple_english), 'English Apple apps')  

There are  9615 English Google apps
There are  6183 English Apple apps


In [16]:
explore_data(apple_english, 0, 5)

['284882215', 'Facebook', '389879808', 'USD', '0.0', '2974676', '212', '3.5', '3.5', '95.0', '4+', 'Social Networking', '37', '1', '29', '1']


['389801252', 'Instagram', '113954816', 'USD', '0.0', '2161558', '1289', '4.5', '4.0', '10.23', '12+', 'Photo & Video', '37', '0', '29', '1']


['529479190', 'Clash of Clans', '116476928', 'USD', '0.0', '2130805', '579', '4.5', '4.5', '9.24.12', '9+', 'Games', '38', '5', '18', '1']


['420009108', 'Temple Run', '65921024', 'USD', '0.0', '1724546', '3842', '4.5', '4.0', '1.6.2', '9+', 'Games', '40', '5', '1', '1']


['284035177', 'Pandora - Music & Radio', '130242560', 'USD', '0.0', '1126879', '3594', '4.0', '4.5', '8.4.1', '12+', 'Music', '37', '4', '1', '1']




Now I will look for free apps. In the Google Play store, free apps are indicated by a 'Price' column (index 7) value of '0', in the Apple store they are indicated by a 'price' column (index 4) value of '0.0'. In the Google Play Store, prices with non-zero values are preceded by a currency character like '$'. However, since all free apps have a price of '0' (a string), I will check for the presence of that string. I am also interested in number of downloads, and how positive the reviews of the app are. In the Apple data, this involves looking at the columns 'rating_count_tot' (that is, total number of ratings), and 'user_rating'. In the Google data, this involves looking at 'Rating' and 'Reviews'. Further information about different columns is in the documentation linked in the introductory paragraph.

In [17]:
print('Before removing non-free items, there were ', len(apple_english), ' Apple apps')
apple_final = []
for app in apple_english:
    price = float(app[4]) #convert the price entries from strings to floats
    if price == 0.0: #if the price of the app is zero
        apple_final.append(app) #add it to the final dataset
print('Afterwards, there are ', len(apple_final), ' Apple apps')
print('\n')
print('Before removing non-free items, there were ', len(google_english), ' Google apps')
google_final = []
for app in google_english:
    price = app[7]
    if price == ('0'):
        google_final.append(app)
print('Afterwards, there are ', len(google_final), ' Google apps')

Before removing non-free items, there were  6183  Apple apps
Afterwards, there are  3222  Apple apps


Before removing non-free items, there were  9615  Google apps
Afterwards, there are  8865  Google apps


## Development strategy

The company proposes a 3-step strategy, to keep development and deployment costs low. First, they wish to develop an Android app and release it in the Google Play Store. If it performs well, they will continue to develop and improve the app. If it continues to do well for six months, they will develop a complementary iOS app for the Apple App Store.

With these goals in mind, it's important to find out what kinds of apps tend to do well in both app stores. First, I will look at the datasets again and think about which columns contain the most relevant data for this question.

In [18]:
print(google_header)
print('\n')
explore_data(google_final, 0, 3)

print(apple_header)
print('\n')
explore_data(apple_final, 0, 3)

['App', 'Category', 'Rating', 'Reviews', 'Size', 'Installs', 'Type', 'Price', 'Content Rating', 'Genres', 'Last Updated', 'Current Ver', 'Android Ver']


['Photo Editor & Candy Camera & Grid & ScrapBook', 'ART_AND_DESIGN', '4.1', '159', '19M', '10,000+', 'Free', '0', 'Everyone', 'Art & Design', 'January 7, 2018', '1.0.0', '4.0.3 and up']


['U Launcher Lite – FREE Live Cool Themes, Hide Apps', 'ART_AND_DESIGN', '4.7', '87510', '8.7M', '5,000,000+', 'Free', '0', 'Everyone', 'Art & Design', 'August 1, 2018', '1.2.4', '4.0.3 and up']


['Sketch - Draw & Paint', 'ART_AND_DESIGN', '4.5', '215644', '25M', '50,000,000+', 'Free', '0', 'Teen', 'Art & Design', 'June 8, 2018', 'Varies with device', '4.2 and up']


['id', 'track_name', 'size_bytes', 'currency', 'price', 'rating_count_tot', 'rating_count_ver', 'user_rating', 'user_rating_ver', 'ver', 'cont_rating', 'prime_genre', 'sup_devices.num', 'ipadSc_urls.num', 'lang.num', 'vpp_lic']


['284882215', 'Facebook', '389879808', 'USD', '0.0', '297

Too see which types of apps are the most popular, I am interested in combining information from the 'Category', 'Installs', 'Rating', and 'Reviews' columns in the Google data. Dataquest suggests also using the 'Genre' column - to me this looks redundant given it appears to mostly duplicate information already present in the 'Category' column, but I will do so and see what happens. In the Apple data, I will look at 'prime_genre', 'user_rating', and 'rating_count_tot' (i.e. the total number of ratings, where more ratings tend to suggest that the 'user_rating' value is representative of the overall sentiment of a large number of people.

First I will write a function to generate frequency tables for individual columns, showing percentages.

In [19]:
def freq_table(data_set, index, percents = False):
    frequencies = {}   #empty dictionary
    
    for row in data_set: #each row is a list relating to a particular app
       
        col = row[index]   #col is the value in the column described by the index value
        
        if col not in frequencies:
            frequencies[col] = 1
        
        elif col in frequencies:
            frequencies[col] += 1
            
    if percents == True: #allows this function to be used to calculate raw frequency numbers or percentages as needed
        total = 0
        for item in frequencies:
            total += frequencies[item] #find total number of items in the list (I know it for these items, but this makes the code generalisable)
        
        for item in frequencies:
            frequencies[item] = round(((frequencies[item]/total) * 100), 2)#change the frequency values in to percentages, rounded to 2 decimal places.

    return(frequencies)

In [20]:
google_categories = freq_table(google_final, 1, False)
google_genres = freq_table(google_final, 9, False)
apple_prime_genres = freq_table(apple_final, 11, False)

Dataquest provided a function which will take the frequency dictionaries as an input and output a table with the dictionaries sorted in descending order by value (raw number or percentage). I have slightly modified the function to allow me to display only the top 'x' results, since a large number of results with small percentages is not particularly informative, and to allow me to display raw frequency numbers or frequencies as percentages of the total number of apps.

In [21]:
def display_table(dataset, index, limit = False, percents = False):
    table = freq_table(dataset, index, percents)
    table_display = []

    for key in table:
        key_val_as_tuple = (table[key], key)
        table_display.append(key_val_as_tuple)
    table_sorted = sorted(table_display, reverse = True)
    
    if  limit == False:
        if percents == False:
             for entry in table_sorted:
                    print(entry[1], ':', entry[0])
        
        elif percents == True:
            for entry in table_sorted:
                print(entry[1], ':', entry[0], '%')
            
    elif limit:
        if percents == False:
            for entry in table_sorted[:limit]:
                print(entry[1], ':', entry[0])
                
        elif percents == True:
            for entry in table_sorted[:limit]:
                print(entry[1], ':', entry[0], '%')

In [22]:
display_table(google_final, index = 1, limit = 10, percents = True) #Google 'Category'

FAMILY : 19.08 %
GAME : 9.66 %
TOOLS : 8.46 %
BUSINESS : 4.58 %
LIFESTYLE : 3.91 %
PRODUCTIVITY : 3.89 %
FINANCE : 3.7 %
MEDICAL : 3.54 %
SPORTS : 3.4 %
PERSONALIZATION : 3.32 %


Looking in the Google Play store, the majority of apps categorized as 'Family' apps seem to be children's games (some educational, some not). This gives the impression that a large plurality of the apps in the app store are games of one type or another.

In [23]:
display_table(google_final, 9, 10, True) #Google 'Genre'

Tools : 8.45 %
Entertainment : 6.07 %
Education : 5.35 %
Business : 4.58 %
Lifestyle : 3.9 %
Productivity : 3.89 %
Finance : 3.7 %
Medical : 3.54 %
Sports : 3.46 %
Personalization : 3.32 %


The 'Genre' column gives a more varied picture. It's difficult to see any particularly strong trends here. Obviously 'Tools' have a larger share than other apps, but not by any huge margin.

In [24]:
display_table(apple_final, 11, 10, True) #Apple 'prime_genre'

Games : 58.16 %
Entertainment : 7.88 %
Photo & Video : 4.97 %
Education : 3.66 %
Social Networking : 3.29 %
Shopping : 2.61 %
Utilities : 2.51 %
Sports : 2.14 %
Music : 2.05 %
Health & Fitness : 2.02 %


The Apple store is very clearly dominated by games. Based on the Google 'Category' column and the Apple genres, it seems that games are the predominant type of app which should appeal to a large audience. However, this tells us only about the apps which have been released - this may indicate more about the ease of developing a certain type of app than how popular those apps actually are. This is also useful information, as we intend to develop apps (and would presumably like that to be as quick and easy as possible), however, the total number of downloads may paint a different picture, so I will look at that next. I will break this down across genres of apps as average rating per app per genre (Apple - they don't give granular download data) and average downloads per app per genre (Google).

In [25]:
#The first step according to Dataquest is to generate a frequency table of the relevant app categorisations. 
# I already did this earlier, they are stored in these variables:
#apple_prime_genres
#google_genres
print("Category: Avg. number of reviews", '\n')
for genre in apple_prime_genres:         #for each unique app genre
    total = 0                                           #initialise a variable for adding up numbers of ratings
    for app in apple_final:                      #for every app in apple_final
        app_genre = app[11]                     #store app genre as a separate variable
        if app_genre == genre:                  #if app genre is the same as the genre we're currently inspecting....
            num_ratings = float(app[5])       #store the number of ratings of that app in a variable
            total += num_ratings                 #add that number of ratings to the total for the genre
    
    len_genre = apple_prime_genres[genre] #length of the genre (number of apps) is stored in the frequency table
    gen_avg = round(total/len_genre)          #average number of ratings per app in the genre (rounded to a whole number)
    print(genre, ': ', gen_avg)

Category: Avg. number of reviews 

Social Networking :  71548
Photo & Video :  28442
Games :  22789
Music :  57327
Reference :  74942
Health & Fitness :  23298
Weather :  52280
Utilities :  18684
Travel :  28244
Shopping :  26920
News :  21248
Navigation :  86090
Lifestyle :  16486
Entertainment :  14030
Food & Drink :  33334
Sports :  23009
Book :  39758
Finance :  31468
Education :  7004
Productivity :  21028
Business :  7491
Catalogs :  4004
Medical :  612


This result is unordered and annoying to look at. Revising the code slightly, using insights from the `display_table()` function:

In [26]:
print("Category: Avg. number of reviews", '\n')
apple_ratings_average = []
for genre in apple_prime_genres: 
    total = 0                       
    for app in apple_final:       
        app_genre = app[11]                    
        if app_genre == genre:                  
            num_ratings = float(app[5])       
            total += num_ratings                
    
    len_genre = apple_prime_genres[genre] 
    gen_avg = round(total/len_genre)          
    genre_tuple = (gen_avg, genre)
    apple_ratings_average.append(genre_tuple)
    
apple_table_sorted = sorted(apple_ratings_average, reverse = True)
for element in apple_table_sorted:
    print(element[1], ':', element[0])
    
    

Category: Avg. number of reviews 

Navigation : 86090
Reference : 74942
Social Networking : 71548
Music : 57327
Weather : 52280
Book : 39758
Food & Drink : 33334
Finance : 31468
Photo & Video : 28442
Travel : 28244
Shopping : 26920
Health & Fitness : 23298
Sports : 23009
Games : 22789
News : 21248
Productivity : 21028
Utilities : 18684
Lifestyle : 16486
Entertainment : 14030
Business : 7491
Education : 7004
Catalogs : 4004
Medical : 612


It appears that by far the most-rated apps are in 'Navigation' and 'Reference' - however, this does not quantify the diversity of the apps in those areas or how much each app contributes to the overall score of the genre.

In [27]:
print("App ----- Number of Reviews", '\n')
for app in apple_final:
    if app[11] == 'Navigation':#if it's a navigation app
        print(app[1], '-----', app[5])     #print the name of the app and number of reviews

App ----- Number of Reviews 

Waze - GPS Navigation, Maps & Real-time Traffic ----- 345046
Google Maps - Navigation & Transit ----- 154911
Geocaching® ----- 12811
CoPilot GPS – Car Navigation & Offline Maps ----- 3582
ImmobilienScout24: Real Estate Search in Germany ----- 187
Railway Route Search ----- 5


There are only 6 apps in the genre, so the large numbers of reviews for Waze and Google Maps are making the genre over-represented in the results (i.e. large number of reviews divided by a small number of apps gives a high average in a small category). How about Reference apps?

In [28]:
print("App ----- Number of Reviews", '\n')
for app in apple_final:
    if app[11] == 'Reference':
        print(app[1], '-----', app[5])

App ----- Number of Reviews 

Bible ----- 985920
Dictionary.com Dictionary & Thesaurus ----- 200047
Dictionary.com Dictionary & Thesaurus for iPad ----- 54175
Google Translate ----- 26786
Muslim Pro: Ramadan 2017 Prayer Times, Azan, Quran ----- 18418
New Furniture Mods - Pocket Wiki & Game Tools for Minecraft PC Edition ----- 17588
Merriam-Webster Dictionary ----- 16849
Night Sky ----- 12122
City Maps for Minecraft PE - The Best Maps for Minecraft Pocket Edition (MCPE) ----- 8535
LUCKY BLOCK MOD ™ for Minecraft PC Edition - The Best Pocket Wiki & Mods Installer Tools ----- 4693
GUNS MODS for Minecraft PC Edition - Mods Tools ----- 1497
Guides for Pokémon GO - Pokemon GO News and Cheats ----- 826
WWDC ----- 762
Horror Maps for Minecraft PE - Download The Scariest Maps for Minecraft Pocket Edition (MCPE) Free ----- 718
VPN Express ----- 14
Real Bike Traffic Rider Virtual Reality Glasses ----- 8
教えて!goo ----- 0
Jishokun-Japanese English Dictionary & Translator ----- 0


This also appears to have an over-representation problem. The Bible and the dictionary are extremely popular. Perhaps a dictionary of Biblical terms would do well. The Quran is also very popular. Maybe an easy-to-use app for another popular religious text (for example the Tanakh + Talmud, the Vedas, the Tao Te Ching, or the Book of Mormon) could do well, especially with an embedded dictionary or glossary of frequently-used terms. What about the Social Networking category?

In [29]:
print("App ----- Number of Reviews", '\n')
for app in apple_final:
    if app[11] == 'Social Networking':
        print(app[1], '-----', app[5])

App ----- Number of Reviews 

Facebook ----- 2974676
Pinterest ----- 1061624
Skype for iPhone ----- 373519
Messenger ----- 351466
Tumblr ----- 334293
WhatsApp Messenger ----- 287589
Kik ----- 260965
ooVoo – Free Video Call, Text and Voice ----- 177501
TextNow - Unlimited Text + Calls ----- 164963
Viber Messenger – Text & Call ----- 164249
Followers - Social Analytics For Instagram ----- 112778
MeetMe - Chat and Meet New People ----- 97072
We Heart It - Fashion, wallpapers, quotes, tattoos ----- 90414
InsTrack for Instagram - Analytics Plus More ----- 85535
Tango - Free Video Call, Voice and Chat ----- 75412
LinkedIn ----- 71856
Match™ - #1 Dating App. ----- 60659
Skype for iPad ----- 60163
POF - Best Dating App for Conversations ----- 52642
Timehop ----- 49510
Find My Family, Friends & iPhone - Life360 Locator ----- 43877
Whisper - Share, Express, Meet ----- 39819
Hangouts ----- 36404
LINE PLAY - Your Avatar World ----- 34677
WeChat ----- 34584
Badoo - Meet New People, Chat, Socialize.

And again the results are largely dominated by the world's most popular social media apps, which is not terribly surprising but also doesn't feel particularly informative. The next category is "Music" - which will presumably be dominated by Apple Music/iTunes and Spotify. Let's see.

In [30]:
print("App ----- Number of Reviews", '\n')
for app in apple_final:
    if app[11] == 'Music':
        print(app[1],'-----', app[5])

App ----- Number of Reviews 

Pandora - Music & Radio ----- 1126879
Spotify Music ----- 878563
Shazam - Discover music, artists, videos & lyrics ----- 402925
iHeartRadio – Free Music & Radio Stations ----- 293228
SoundCloud - Music & Audio ----- 135744
Magic Piano by Smule ----- 131695
Smule Sing! ----- 119316
TuneIn Radio - MLB NBA Audiobooks Podcasts Music ----- 110420
Amazon Music ----- 106235
SoundHound Song Search & Music Player ----- 82602
Sonos Controller ----- 48905
Bandsintown Concerts ----- 30845
Karaoke - Sing Karaoke, Unlimited Songs! ----- 28606
My Mixtapez Music ----- 26286
Sing Karaoke Songs Unlimited with StarMaker ----- 26227
Ringtones for iPhone & Ringtone Maker ----- 25403
Musi - Unlimited Music For YouTube ----- 25193
AutoRap by Smule ----- 18202
Spinrilla - Mixtapes For Free ----- 15053
Napster - Top Music & Radio ----- 14268
edjing Mix:DJ turntable to remix and scratch music ----- 13580
Free Music - MP3 Streamer & Playlist Manager Pro ----- 13443
Free Piano app by

I didn't expect Pandora to be so popular, but in essence the problem is the same. Extremely famous and popular apps dominate the market and have large numbers of reviews. This over-representation greatly decreases the value of looking at categories based on average number of reviews - unfortunately we can't make a business model by simply creating an app which is already famous and popular.

The Apple data lead me to conclude that a simple-to-use app which contains the text of a very popular book along with a dictionary/glossary related to the text could perform quite well. A benefit of this is that popular religious texts are almost certainly in the public domain and therefore free to reproduce.

I will now look at Google data to see if I can find out anything more useful. Dataquest suggests using information about the number of downloads (which is not particularly granular), but first I will check average number of reviews per category, out of interest.

In [31]:
#Rewriting the above code to display review data from the Google App Store by Categories:
print("Category: Avg. number of reviews", '\n')
google_ratings_average = []
for category in google_categories: 
    total = 0                       
    for app in google_final:       
        app_category = app[1]                    
        if app_category == category:                  
            num_ratings = float(app[3])       
            total += num_ratings                
    
    len_category = google_categories[category] 
    cat_avg = round(total/len_category)          
    category_tuple = (cat_avg, category)
    google_ratings_average.append(category_tuple)
    
google_table_sorted = sorted(google_ratings_average, reverse = True)
for element in google_table_sorted:
    print(element[1], ':', element[0])
    

Category: Avg. number of reviews 

COMMUNICATION : 992212
SOCIAL : 965644
GAME : 680264
VIDEO_PLAYERS : 427057
PHOTOGRAPHY : 404062
TOOLS : 305687
ENTERTAINMENT : 256156
SHOPPING : 223836
PERSONALIZATION : 181109
WEATHER : 171237
PRODUCTIVITY : 160619
MAPS_AND_NAVIGATION : 142788
TRAVEL_AND_LOCAL : 129484
FAMILY : 119445
SPORTS : 116937
NEWS_AND_MAGAZINES : 93087
BOOKS_AND_REFERENCE : 87995
HEALTH_AND_FITNESS : 77184
FOOD_AND_DRINK : 57479
EDUCATION : 56891
COMICS : 42586
FINANCE : 38536
LIFESTYLE : 33822
HOUSE_AND_HOME : 26435
ART_AND_DESIGN : 24699
BUSINESS : 23834
DATING : 21953
PARENTING : 16379
AUTO_AND_VEHICLES : 14140
LIBRARIES_AND_DEMO : 10926
BEAUTY : 7476
MEDICAL : 4035
EVENTS : 2556


In [32]:
print("App ------- number of reviews", '\n')
communication_apps =[]
for app in google_final:
    if app[1] == 'COMMUNICATION':
        app_tuple = (float(app[3]), app[0])
        communication_apps.append(app_tuple)
            
communication_sorted = sorted(communication_apps, reverse = True)
for element in communication_sorted:
    print(element[1], element[0])

App ------- number of reviews 

WhatsApp Messenger 69119316.0
Messenger – Text and Video Chat for Free 56646578.0
UC Browser - Fast Download Private & Secure 17714850.0
BBM - Free Calls & Messages 12843436.0
Viber Messenger 11335481.0
LINE: Free Calls & Messages 10790289.0
Skype - free IM & video calls 10484169.0
Google Chrome: Fast & Secure 9643041.0
Truecaller: Caller ID, SMS spam blocking & Dialer 7820775.0
WeChat 5387631.0
Opera Mini - fast web browser 5150801.0
imo free video calls and chat 4785988.0
Gmail 4604483.0
Yahoo Mail – Stay Organized 4188345.0
UC Browser Mini -Tiny Fast Private & Secure 3648765.0
Hangouts 3419513.0
Telegram 3128611.0
Firefox Browser fast & private 3075118.0
GO SMS Pro - Messenger, Free Themes, Emoji 2876500.0
KakaoTalk: Free Calls & Text 2546549.0
Dolphin Browser - Fast, Private & Adblock🐬 2511130.0
Opera Browser: Fast and Secure 2473795.0
Kik 2451136.0
Who 2451093.0
CM Browser - Ad Blocker , Fast Download , Privacy 2265084.0
Google Duo - High Quality Vi

Unsurprisingly, this still doesn't give much insight into how to create an app which will have moderate success in making money - unless the business model is to simply produce clones of the most popular apps in the world and (hopefully) not get sued.

I'll try what Dataquest suggested:

In [33]:
#First they ask me to create a frequency table of categories - I already did this as google_categories earlier.
#They then ask me to loop over the google_final dataset to calculate the average number of installs per app per category.
#Dataquest instructs me to initialise a variable to count apps and calculate the lengths of categories.
# I will not add this, as the information is already present in the google_categories frequency table (it has absolute counts rather than %s)


#I will rank the categories by average number of installs for ease of consideration.

categories_unsorted = []

for category in google_categories:
        total = 0
        for app in google_final:
            if app[1] == category:               #if the category of the app is the category of interest
                num_installs = app[5]
                num_installs = num_installs.replace('+', '')
                num_installs = num_installs.replace(',', '')
                num_installs = float(num_installs)
                total += num_installs
                
        average = total/google_categories[category]
        category_tuple = (round(average), category)
        categories_unsorted.append(category_tuple)

categories_sorted = sorted(categories_unsorted, reverse = True)
for category in categories_sorted:
    print(category[1], ':', category[0])

COMMUNICATION : 38326063
VIDEO_PLAYERS : 24821087
SOCIAL : 23253652
PHOTOGRAPHY : 17840110
PRODUCTIVITY : 16772839
GAME : 15481156
TRAVEL_AND_LOCAL : 13984078
TOOLS : 10801391
ENTERTAINMENT : 10296962
NEWS_AND_MAGAZINES : 9549178
BOOKS_AND_REFERENCE : 8767812
SHOPPING : 7036877
PERSONALIZATION : 5201483
WEATHER : 5074486
HEALTH_AND_FITNESS : 4167457
MAPS_AND_NAVIGATION : 4056942
FAMILY : 3883439
SPORTS : 3638640
ART_AND_DESIGN : 1986335
FOOD_AND_DRINK : 1924898
EDUCATION : 1850000
BUSINESS : 1704192
LIFESTYLE : 1433676
FINANCE : 1387692
HOUSE_AND_HOME : 1331541
DATING : 854029
COMICS : 817657
AUTO_AND_VEHICLES : 647318
LIBRARIES_AND_DEMO : 638504
PARENTING : 542604
BEAUTY : 513152
EVENTS : 253542
MEDICAL : 123065


I suspect that many of the top categories have the same problem as the Apple dataset - that is, particularly popular apps lead to a high average for their entire category and therefore lead to it being over-represented in the results. This ultimately means that the types/category of apps which are, in general, most popular, will not actually show up at the top. I'll check the first few categories to see:

In [34]:
for app in google_final:
    if app[1] == "COMMUNICATION":
        print(app[0], ':', app[5])

WhatsApp Messenger : 1,000,000,000+
Messenger for SMS : 10,000,000+
My Tele2 : 5,000,000+
imo beta free calls and text : 100,000,000+
Contacts : 50,000,000+
Call Free – Free Call : 5,000,000+
Web Browser & Explorer : 5,000,000+
Browser 4G : 10,000,000+
MegaFon Dashboard : 10,000,000+
ZenUI Dialer & Contacts : 10,000,000+
Cricket Visual Voicemail : 10,000,000+
TracFone My Account : 1,000,000+
Xperia Link™ : 10,000,000+
TouchPal Keyboard - Fun Emoji & Android Keyboard : 10,000,000+
Skype Lite - Free Video Call & Chat : 5,000,000+
My magenta : 1,000,000+
Android Messages : 100,000,000+
Google Duo - High Quality Video Calls : 500,000,000+
Seznam.cz : 1,000,000+
Antillean Gold Telegram (original version) : 100,000+
AT&T Visual Voicemail : 10,000,000+
GMX Mail : 10,000,000+
Omlet Chat : 10,000,000+
My Vodacom SA : 5,000,000+
Messenger – Text and Video Chat for Free : 1,000,000,000+
imo free video calls and chat : 500,000,000+
Calls & Text by Mo+ : 5,000,000+
free video calls and chat : 50,00

Some of the apps with the largest number of installs include Whatsapp and Facebook's Messenger. Others include internet browsers and a variety of apps like voicemail which tend to come pre-installed on phones out of the box.

In [35]:
for app in google_final:
    if app[1] == "VIDEO_PLAYERS":
        print(app[0], ':', app[5])

All Video Downloader 2018 : 1,000,000+
HD Video Player : 1,000,000+
Iqiyi (for tablet) : 1,000,000+
Motorola Gallery : 100,000,000+
Free TV series : 100,000+
Video Player All Format for Android : 500,000+
Code : 10,000,000+
Vote for : 50,000,000+
XX HD Video downloader-Free Video Downloader : 1,000,000+
OBJECTIVE : 1,000,000+
Music - Mp3 Player : 10,000,000+
HD Movie Video Player : 1,000,000+
YouCut - Video Editor & Video Maker, No Watermark : 5,000,000+
Video Editor,Crop Video,Movie Video,Music,Effects : 1,000,000+
video player for android : 10,000,000+
Google Play Movies & TV : 1,000,000,000+
HTC Service － DLNA : 10,000,000+
VPlayer : 1,000,000+
MiniMovie - Free Video and Slideshow Editor : 50,000,000+
Samsung Video Library : 50,000,000+
OnePlus Gallery : 1,000,000+
LIKE – Magic Video Maker & Community : 50,000,000+
HTC Service—Video Player : 5,000,000+
Play Tube : 1,000,000+
Droid Zap by Motorola : 5,000,000+
video player : 1,000,000+
G Guide Program Guide (SOFTBANK EMOBILE WILLCOM 

The main players here seem to be mainly screen-recording apps. These are presumably popular for sharing videos between apps with no easy interoperability (for example screencapping TikTok videos to share as Instagram reels). A good screen capture app could do well, although the market seems quite saturated. Video editor apps are also quite popular and could serve similar functions (for example adding watermarks to recorded videos, as may be done by meme pages).

In [36]:
for app in google_final:
    if app[1] == "SOCIAL":
        print(app[0], ':', app[5])

Social network all in one 2018 : 100,000+
Pinterest : 100,000,000+
TextNow - free text + calls : 10,000,000+
The Messenger App : 1,000,000+
Messenger Pro : 1,000,000+
Free Messages, Video, Chat,Text for Messenger Plus : 1,000,000+
The Video Messenger App : 100,000+
Jodel - The Hyperlocal App : 1,000,000+
Hide Something - Photo, Video : 5,000,000+
Love Sticker : 1,000,000+
Web Browser & Fast Explorer : 5,000,000+
VidStatus app - Status Videos & Status Downloader : 5,000,000+
Love Images : 1,000,000+
Web Browser ( Fast & Secure Web Explorer) : 500,000+
SPARK - Live random video chat & meet new people : 5,000,000+
Golden telegram : 50,000+
Facebook Local : 1,000,000+
Meet – Talk to Strangers Using Random Video Chat : 5,000,000+
MobilePatrol Public Safety App : 1,000,000+
💘 WhatsLov: Smileys of love, stickers and GIF : 1,000,000+
HTC Social Plugin - Facebook : 10,000,000+
Kate Mobile for VK : 10,000,000+
Family GPS tracker KidControl + GPS by SMS Locator : 1,000,000+
Moment : 1,000,000+
Te

This category contains large numbers of messaging and dating apps. The most popular apps have very large numbers of downloads, this is of course inherent to being popular and well-known. Designing a very popular social media app would almost certainly require a serious investment of time and large amounts of advertising, given that the users of sites like Facbook and Instagram would likely be reluctant to join sites which their friends/contacts are not also joining. 

Looking at the various other categories, BOOKS_AND_REFERENCE is ranked reasonably well here as well, though it's not right at the top. Categories like photography and travel would likely require larger investments of time (photography apps often imply a social/sharing component, and most phones already have one pre-installed anyway; travel apps would probably require a lot of research). 

Given these results, I think that an app which contains the text of a popular (free, out of copyright) book (especially a religious or philosophical text) with some kind of reference dictionary/glossary could do reasonably well without any huge investment of time or resources. Adding the ability to highlight and store favourite quotes would probably be beneficial, as would some sort of incentive to repeatedly return to the app to create ad impressions (for example reading goals/reminders, the option to share favourite quotes over social media/text, and "daily wisdom"-style quotes of the text).